# 지도학습_분류분석_컬럼선택법_클래스불균형_모델성능향상시키기

# 중요 컬럼 선택하기
* 1) EDA를 통해서 종속변수(target)과 중요한 관계가 있는 변수들만 선택
* 2) 수치형 변수는 상관분석 결과를 통해, 범주형 변수는 카이제곱 통계량 (통계학적 방법)
* 3) 머신러닝 알고리즘 통한 1차 분석 후 중요하게 사용된 변수만 선택

# tree 계열 모델의 feature_importance로 선택하기

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

In [3]:
data = pd.read_csv("./data/salary2.csv")
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [4]:
data2 = data.copy()

In [6]:
data = data.drop_duplicates()

In [7]:
data = data.dropna()
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48835,53,Private,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
48836,22,Private,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [9]:
data = data.reset_index(drop=True)
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39235,53,Private,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
39236,22,Private,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
39237,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
39238,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


In [12]:
data['class'] = data['class'].apply(lambda x: 1 if x == ' >50K' else 0)

In [14]:
data = pd.get_dummies(data, drop_first = True)

In [16]:
X = data.drop('class', axis=1)
y = data['class']

In [17]:
X

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,25,7,0,0,40,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,38,9,0,0,50,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,28,12,0,0,40,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,44,10,7688,0,40,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,34,6,0,0,30,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39235,53,14,0,0,40,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
39236,22,10,0,0,40,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
39237,27,12,0,0,38,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
39238,58,9,0,0,40,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [18]:
y

0        0
1        0
2        1
3        1
4        0
        ..
39235    1
39236    0
39237    0
39238    0
39239    1
Name: class, Length: 39240, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=y, random_state=10)

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [23]:
dtc = DecisionTreeClassifier(max_depth = 9, random_state=10)
dtc.fit(X_train, y_train)
pred = dtc.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.87      0.94      0.90     11706
           1       0.77      0.58      0.66      3990

    accuracy                           0.85     15696
   macro avg       0.82      0.76      0.78     15696
weighted avg       0.84      0.85      0.84     15696



# feature_importance 출력하기

In [28]:
importance = pd.DataFrame(dtc.feature_importances_)

In [29]:
cols = pd.DataFrame(dtc.feature_names_in_)

In [33]:
important_cols = pd.concat([cols, importance], axis=1)

In [34]:
important_cols

,0,0
0,age,0.038730
1,education-num,0.194695
2,capital-gain,0.216183
3,capital-loss,0.078845
4,hours-per-week,0.031110
...,...,...
90,native-country_ Thailand,0.000000
91,native-country_ Trinadad&Tobago,0.000000
92,native-country_ United-States,0.000000
93,native-country_ Vietnam,0.000000


In [35]:
important_cols.columns = ['column', 'importance']

In [36]:
important_cols

,column,importance
0,age,0.038730
1,education-num,0.194695
2,capital-gain,0.216183
3,capital-loss,0.078845
4,hours-per-week,0.031110
...,...,...
90,native-country_ Thailand,0.000000
91,native-country_ Trinadad&Tobago,0.000000
92,native-country_ United-States,0.000000
93,native-country_ Vietnam,0.000000


In [38]:
important_cols = important_cols.sort_values(by='importance', ascending=False)

In [39]:
important_cols[important_cols['importance'] > 0]

,column,importance
27,marital-status_ Married-civ-spouse,0.400573
2,capital-gain,0.216183
1,education-num,0.194695
3,capital-loss,0.078845
0,age,0.038730
4,hours-per-week,0.031110
34,occupation_ Exec-managerial,0.011520
8,workclass_ Self-emp-not-inc,0.006948
35,occupation_ Farming-fishing,0.004866
54,sex_ Male,0.003542


# random_forest

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
importance2 = pd.DataFrame(rfc.feature_importances_, columns=['importance'])

In [44]:
column2 = pd.DataFrame(rfc.feature_names_in_, columns=['columns'])

In [46]:
rfc_importance = pd.concat([column2, importance2], axis=1)

In [47]:
rfc_importance

,columns,importance
0,age,0.229726
1,education-num,0.060181
2,capital-gain,0.122142
3,capital-loss,0.041190
4,hours-per-week,0.117826
...,...,...
90,native-country_ Thailand,0.000140
91,native-country_ Trinadad&Tobago,0.000107
92,native-country_ United-States,0.006653
93,native-country_ Vietnam,0.000283


In [53]:
rfc_importance = rfc_importance[rfc_importance['importance'] > 0.01].sort_values(by='importance', ascending=False)

In [54]:
rfc_importance

,columns,importance
0,age,0.229726
2,capital-gain,0.122142
4,hours-per-week,0.117826
27,marital-status_ Married-civ-spouse,0.083482
1,education-num,0.060181
3,capital-loss,0.041190
29,marital-status_ Never-married,0.034063
34,occupation_ Exec-managerial,0.019778
54,sex_ Male,0.019321
45,relationship_ Not-in-family,0.017576


In [55]:
from xgboost import XGBClassifier 

In [56]:
xgb = XGBClassifier(random_state = 10)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91     11706
           1       0.77      0.67      0.71      3990

    accuracy                           0.86     15696
   macro avg       0.83      0.80      0.81     15696
weighted avg       0.86      0.86      0.86     15696



In [68]:
importance3 = pd.DataFrame(xgb.feature_importances_, columns=['importance'])
column3 = pd.DataFrame(xgb.feature_names_in_, columns=['column'])
importance3 = pd.concat([column3, importance3], axis=1)

In [69]:
importance3=importance3[importance3['importance'] > 0.01].sort_values(by='importance', ascending=False)

In [70]:
importance3

,column,importance
27,marital-status_ Married-civ-spouse,0.413689
2,capital-gain,0.058502
1,education-num,0.046971
38,occupation_ Other-service,0.033853
3,capital-loss,0.024621
35,occupation_ Farming-fishing,0.023159
34,occupation_ Exec-managerial,0.021343
47,relationship_ Own-child,0.015753
40,occupation_ Prof-specialty,0.013670
79,native-country_ Mexico,0.013073


In [89]:
data2.columns

Index(['age', 'workclass', 'education', 'education-num', 'marital-status',
       'occupation', 'relationship', 'race', 'sex', 'capital-gain',
       'capital-loss', 'hours-per-week', 'native-country', 'class'],
      dtype='object')

In [101]:
data2 = data2.drop_duplicates()
data2 = data2.dropna()
data3 = data2[['marital-status', 'capital-gain', 'capital-loss', 'sex', 'education-num', 'occupation', 'class', 'age']]

In [102]:
data = data3.drop_duplicates()

data3 = data3.dropna()


In [103]:
data3

,marital-status,capital-gain,capital-loss,sex,education-num,occupation,class,age
0,Never-married,0,0,Male,7,Machine-op-inspct,<=50K,25
1,Married-civ-spouse,0,0,Male,9,Farming-fishing,<=50K,38
2,Married-civ-spouse,0,0,Male,12,Protective-serv,>50K,28
3,Married-civ-spouse,7688,0,Male,10,Machine-op-inspct,>50K,44
5,Never-married,0,0,Male,6,Other-service,<=50K,34
...,...,...,...,...,...,...,...,...
48835,Married-civ-spouse,0,0,Male,14,Exec-managerial,>50K,53
48836,Never-married,0,0,Male,10,Protective-serv,<=50K,22
48837,Married-civ-spouse,0,0,Female,12,Tech-support,<=50K,27
48839,Widowed,0,0,Female,9,Adm-clerical,<=50K,58


In [104]:
data3 = data3.reset_index(drop=True)
data3

,marital-status,capital-gain,capital-loss,sex,education-num,occupation,class,age
0,Never-married,0,0,Male,7,Machine-op-inspct,<=50K,25
1,Married-civ-spouse,0,0,Male,9,Farming-fishing,<=50K,38
2,Married-civ-spouse,0,0,Male,12,Protective-serv,>50K,28
3,Married-civ-spouse,7688,0,Male,10,Machine-op-inspct,>50K,44
4,Never-married,0,0,Male,6,Other-service,<=50K,34
...,...,...,...,...,...,...,...,...
39235,Married-civ-spouse,0,0,Male,14,Exec-managerial,>50K,53
39236,Never-married,0,0,Male,10,Protective-serv,<=50K,22
39237,Married-civ-spouse,0,0,Female,12,Tech-support,<=50K,27
39238,Widowed,0,0,Female,9,Adm-clerical,<=50K,58


In [105]:
data3 = pd.get_dummies(data3, drop_first=True)

In [106]:
data3

,capital-gain,capital-loss,education-num,age,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,...,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,class_ >50K
0,0,0,7,25,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
1,0,0,9,38,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0,0,12,28,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3,7688,0,10,44,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
4,0,0,6,34,False,False,False,True,False,False,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39235,0,0,14,53,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
39236,0,0,10,22,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
39237,0,0,12,27,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
39238,0,0,9,58,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [107]:
X = data3.drop('class_ >50K', axis=1)
y = data3['class_ >50K']

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=y, random_state=10)

In [109]:
xgb = XGBClassifier(random_state=10)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

       False       0.89      0.93      0.91     11706
        True       0.77      0.65      0.70      3990

    accuracy                           0.86     15696
   macro avg       0.83      0.79      0.81     15696
weighted avg       0.86      0.86      0.86     15696



In [110]:
importance3 = pd.DataFrame(xgb.feature_importances_, columns=['importance'])
column3 = pd.DataFrame(xgb.feature_names_in_, columns=['column'])
importance3 = pd.concat([column3, importance3], axis=1)
importance3=importance3[importance3['importance'] > 0.01].sort_values(by='importance', ascending=False)
importance3

,column,importance
5,marital-status_ Married-civ-spouse,0.473459
0,capital-gain,0.090947
2,education-num,0.057546
17,occupation_ Other-service,0.056346
14,occupation_ Farming-fishing,0.047047
13,occupation_ Exec-managerial,0.039511
1,capital-loss,0.035579
19,occupation_ Prof-specialty,0.028145
22,occupation_ Tech-support,0.021765
15,occupation_ Handlers-cleaners,0.017977


# 클래스 불균형을 처리하는 방법
* 클래스 불균형: 종속변수(target)의 클래스(값, 요소)의 비율이 일치하지 않는 경우
* 연봉 데이터의 경우 50K 초과가 1, 50K 이하가 3인 비율을 가지고 있음
* train_test_split시 stratify = 옵션을 반드시 포함
* 머신러닝 알고리즘에 있는 소수 클래스에 가중치를 주는 옵션 on

decisionTree 모델의 경우 class_weight으로 balanced 주면 됨

In [112]:
for i in range(1, 20):

    dtc = DecisionTreeClassifier(max_depth=i, class_weight="balanced", random_state=10)
    dtc.fit(X_train, y_train)
    pred = dtc.predict(X_test)
    print(i)
    print(classification_report(y_test, pred))
    print()

1
              precision    recall  f1-score   support

       False       0.92      0.67      0.78     11706
        True       0.47      0.83      0.60      3990

    accuracy                           0.71     15696
   macro avg       0.69      0.75      0.69     15696
weighted avg       0.81      0.71      0.73     15696


2
              precision    recall  f1-score   support

       False       0.94      0.67      0.78     11706
        True       0.48      0.88      0.62      3990

    accuracy                           0.72     15696
   macro avg       0.71      0.77      0.70     15696
weighted avg       0.82      0.72      0.74     15696


3
              precision    recall  f1-score   support

       False       0.89      0.85      0.87     11706
        True       0.62      0.70      0.66      3990

    accuracy                           0.81     15696
   macro avg       0.76      0.78      0.76     15696
weighted avg       0.82      0.81      0.82     15696


4
        

random_forest의 경우 class_weight="balanced"

In [115]:
for i in range(1, 20):

    rfc = RandomForestClassifier(max_depth=i, n_estimators=100 ,n_jobs=-1, class_weight="balanced", random_state=10)
    rfc.fit(X_train, y_train)
    pred = rfc.predict(X_test)
    print(i)
    print(classification_report(y_test, pred))
    print()

1
              precision    recall  f1-score   support

       False       0.93      0.70      0.80     11706
        True       0.49      0.85      0.62      3990

    accuracy                           0.74     15696
   macro avg       0.71      0.77      0.71     15696
weighted avg       0.82      0.74      0.75     15696


2
              precision    recall  f1-score   support

       False       0.94      0.70      0.80     11706
        True       0.50      0.87      0.63      3990

    accuracy                           0.75     15696
   macro avg       0.72      0.79      0.72     15696
weighted avg       0.83      0.75      0.76     15696


3
              precision    recall  f1-score   support

       False       0.94      0.71      0.81     11706
        True       0.51      0.87      0.64      3990

    accuracy                           0.75     15696
   macro avg       0.72      0.79      0.72     15696
weighted avg       0.83      0.75      0.77     15696


4
        

xgboost scale_pos_weight=3

In [117]:
for i in range(1, 21):
        
    xgb = XGBClassifier(max_depth=i, n_estimators=500, scale_pos_weight=3, n_jobs=-1, random_state=10)
    xgb.fit(X_train, y_train)
    pred = xgb.predict(X_test)
    print(i)
    print(classification_report(y_test, pred))
    print()

1
              precision    recall  f1-score   support

       False       0.94      0.81      0.87     11706
        True       0.60      0.85      0.70      3990

    accuracy                           0.82     15696
   macro avg       0.77      0.83      0.79     15696
weighted avg       0.85      0.82      0.83     15696


2
              precision    recall  f1-score   support

       False       0.94      0.81      0.87     11706
        True       0.61      0.86      0.71      3990

    accuracy                           0.82     15696
   macro avg       0.78      0.84      0.79     15696
weighted avg       0.86      0.82      0.83     15696


3
              precision    recall  f1-score   support

       False       0.94      0.81      0.87     11706
        True       0.61      0.86      0.71      3990

    accuracy                           0.82     15696
   macro avg       0.77      0.83      0.79     15696
weighted avg       0.86      0.82      0.83     15696


4
        

lightgbm is_unbalance = True

In [118]:
from lightgbm import LGBMClassifier

In [119]:
for i in range(1, 21):
        
    lgbm = LGBMClassifier(max_depth=i, n_estimators=500, is_unbalanced=True, n_jobs=-1, random_state=10)
    lgbm.fit(X_train, y_train)
    pred = lgbm.predict(X_test)
    print(i)
    print(classification_report(y_test, pred))
    print()

[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Info] Number of positive: 5985, number of negative: 17559
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000638 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 288
[LightGBM] [Info] Number of data points in the train set: 23544, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.254205 -> initscore=-1.076310
[LightGBM] [Info] Start training from score -1.076310
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

              precision    recall  f1-score   support

       False       0.87      0.94      0.90     11706
        True       0.77      0.58      0.66      3990

    accuracy                           0.85     15696
   macro avg       0.82      0.76      0.78     15696
weighted avg       0.84      0.85      0.84     15696


[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Info] Number of positive: 5985, number of negative: 17559
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001132 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 288
[LightGBM] [Info] Number of data points in the train set: 23544, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.254205 -> initscore=-1.076310
[LightGBM] [Info] Start training from score -1

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Unknown parameter: is_unbalanced
10
              precision    recall  f1-score   support

       False       0.88      0.93      0.91     11706
        True       0.76      0.64      0.69      3990

    accuracy                           0.86     15696
   macro avg       0.82      0.78      0.80     15696
weighted avg       0.85      0.86      0.85     15696


[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Info] Number of positive: 5985, number of negative: 17559
[LightGBM] [Info] Auto-choosing col-wise multi-threading,

[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: is_unbalanced
[LightGBM] [Info] Number of positive: 5985, number of negative: 17559
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000868 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 288
[LightGBM] [Info] Number of data points in the train set: 23544, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.254205 -> initscore=-1.076310
[LightGBM] [Info] Start training from score -1.076310
[LightGBM] [Warning] Unknown parameter: is_unbalanced
18
              precision    recall  f1-score   support

       False       0.88      0.93      0.91     11706
        True       0.76      0.64      0.69      3990

    accuracy                           0.86     15696
   macro avg       0.82      0.78      0.80  

# 데이터를 증폭/축소해서 class 불균형 해소
* imbalanced-learn, smote/cluster-centroids
* 머신러닝 알고리즘을 통해 비슷한 데이터를 생성/축소
* **반드시 train 데이터에만 적용**

In [120]:
list(range(4, X_train.shape[1]))

,capital-gain,capital-loss,education-num,age,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,...,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving
21251,0,0,4,32,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3994,0,0,10,38,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1871,0,0,13,58,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
16563,0,0,9,27,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
16892,0,0,13,46,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15385,0,1340,13,25,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
37453,0,0,5,34,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
14106,0,0,9,25,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
20368,0,0,9,70,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [121]:
y_train

21251    False
3994     False
1871      True
16563    False
16892     True
         ...  
15385    False
37453    False
14106    False
20368    False
36269    False
Name: class_ >50K, Length: 23544, dtype: bool

In [123]:
X_train.shape

(23544, 24)

In [125]:
from imblearn.over_sampling import SMOTENC
smt = SMOTENC(categorical_features=list(range(4, X_train.shape[1])), random_state=10, k_neighbors=5, n_jobs=-1)
smt_X, smt_y = smt.fit_resample(X_train, y_train)

C:\Users\vbak0\miniforge3\envs\fintech\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
C:\Users\vbak0\miniforge3\envs\fintech\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
C:\Users\vbak0\miniforge3\envs\fintech\lib\site-packages\imblearn\over_sampling\_smote\base.py:645: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


KeyboardInterrupt: 

In [124]:
dtc2 = DecisionTreeClassifier(max_depth=i, random_state=10)
dtc2.fit(smt_X, smt_y)
pred = dtc2.predict(X_test)
print(i)
print(classification_report(y_test, pred))

NameError: name 'smt_X' is not defined

# k-fold, 하이퍼파라미터 튜닝
* k-fold 교차검증: 훈련에 사용할 데이터를 여러개로 나누어서 편향을 줄이고 모델을 균형 잡히게 하는 방법
* 하이퍼파라미터 튜닝: 머신러닝 알고리즘에 있는 여러 파라미터를 조정해서 최고의 성능을 내는 모델을 내는 것 **max_depth, n_estimators, learning_rate**
* gridsearch, randomsearch
* gridsearch: 지정해 놓은 모든 경우의 수의 하이퍼파라미터를 탐색
* randomsearch: 지정해 놓은 범위 내에서 랜덤하게 탐색

In [127]:
from sklearn.model_selection import GridSearchCV

In [128]:
from sklearn.tree import DecisionTreeClassifier

In [130]:
model = DecisionTreeClassifier()

In [134]:
params = dict(criterion=["gini", "entropy", "log_loss"],
              max_depth=[1, 5, 10, 15],
              random_state=[7, 10, 777],
              class_weight=[None, 'balanced'])

In [135]:
grid_cv = GridSearchCV(model, param_grid=params, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)
pred = grid_cv.predict(X_test)
print("best_params: ", grid_cv.best_params_)
print("best_score_: ", grid_cv.best_score_)

best_params:  {'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'random_state': 7}
best_score_:  0.8496856948691812


In [137]:
model2 = DecisionTreeClassifier(criterion= 'gini', max_depth= 10, random_state= 7)
model2.fit(X_train, y_train)
pred = model2.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

       False       0.87      0.94      0.90     11706
        True       0.76      0.57      0.66      3990

    accuracy                           0.85     15696
   macro avg       0.81      0.76      0.78     15696
weighted avg       0.84      0.85      0.84     15696



In [138]:
params = dict(criterion=["gini", "entropy", "log_loss"],
              max_depth=[8, 9, 10, 11, 12],
              random_state=[7],
              class_weight=[None, 'balanced'])

In [139]:
grid_cv = GridSearchCV(model, param_grid=params, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)
pred = grid_cv.predict(X_test)
print("best_params: ", grid_cv.best_params_)
print("best_score_: ", grid_cv.best_score_)

best_params:  {'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'random_state': 7}
best_score_:  0.8496856948691812


gridSearch에서 score 기준 변경하기 
* scoring=[accuracy, precision, racall, f1, roc_auc]

In [140]:
grid_cv = GridSearchCV(model, param_grid=params, scoring='recall', cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)
pred = grid_cv.predict(X_test)
print("best_params: ", grid_cv.best_params_)
print("best_score_: ", grid_cv.best_score_)

best_params:  {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 10, 'random_state': 7}
best_score_:  0.8621553884711779


In [141]:
model3 = DecisionTreeClassifier(criterion= 'gini', max_depth= 10, random_state= 7, class_weight='balanced')
model3.fit(X_train, y_train)
pred = model3.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

       False       0.95      0.76      0.84     11706
        True       0.55      0.87      0.68      3990

    accuracy                           0.79     15696
   macro avg       0.75      0.82      0.76     15696
weighted avg       0.85      0.79      0.80     15696



클래스 0에 대한 score를 계산하고 싶을 때

In [143]:
from sklearn.metrics import make_scorer, recall_score, precision_score, accuracy_score

In [144]:
recall_0_scorer = make_scorer(recall_score, pos_label=0)

In [146]:
grid_cv = GridSearchCV(model, param_grid=params, scoring=recall_0_scorer, cv=3, n_jobs=-1)
grid_cv.fit(X_train, y_train)
pred = grid_cv.predict(X_test)
print("best_params: ", grid_cv.best_params_)
print("best_score_: ", grid_cv.best_score_)

best_params:  {'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'random_state': 7}
best_score_:  0.9421379349621278


In [147]:
model3 = DecisionTreeClassifier(criterion= 'gini', max_depth= 10, random_state= 7)
model3.fit(X_train, y_train)
pred = model3.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

       False       0.87      0.94      0.90     11706
        True       0.76      0.57      0.66      3990

    accuracy                           0.85     15696
   macro avg       0.81      0.76      0.78     15696
weighted avg       0.84      0.85      0.84     15696



# randomSearch
* 지정한 파라미터 값에서 랜덤성을 더해서 최적 값을 찾아냄

In [148]:
from sklearn.model_selection import RandomizedSearchCV

In [149]:
dtc = DecisionTreeClassifier(random_state=10)

In [151]:
params = dict(criterion=["gini", "entropy", "log_loss"],
              max_depth=[1, 5, 10, 15],
              random_state=[7, 10, 777],
              class_weight=[None, 'balanced'])

In [152]:
rand_cv = RandomizedSearchCV(model, param_distributions=params, scoring='f1',
                            cv=3, n_jobs=-1, random_state=10)

rand_cv.fit(X_train, y_train)
pred = rand_cv.predict(X_test)
print("best_params: ", rand_cv.best_params_)
print("best_score_: ", rand_cv.best_score_)


best_params:  {'random_state': 7, 'max_depth': 15, 'criterion': 'log_loss', 'class_weight': 'balanced'}
best_score_:  0.6839898770853573


In [153]:
model4 = DecisionTreeClassifier(random_state= 7, max_depth= 15, criterion='entropy', class_weight= 'balanced')
model4.fit(X_train, y_train)
pred = model4.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

       False       0.94      0.78      0.85     11706
        True       0.56      0.85      0.68      3990

    accuracy                           0.79     15696
   macro avg       0.75      0.81      0.76     15696
weighted avg       0.84      0.79      0.80     15696



XGBoost를 랜덤서치로 하이퍼파라미터 튜닝하기

In [154]:
xgb = XGBClassifier(n_jobs=-1, random_state=10)

In [155]:
xgb_params = dict(n_estimators=[100, 500, 1000],
                 max_depth = [1, 5, 10, 15, 20],
                 learning_rate=[0.1, 0.5, 1],
                 booster=['gbtree', 'gblinear', 'dart'],
                 scale_pos_weight=[0, 3])

In [156]:
rand_cv = RandomizedSearchCV(xgb, param_distributions=xgb_params, cv=3, scoring='roc_auc', random_state=10)

rand_cv.fit(X_train, y_train)
pred = rand_cv.predict(X_test)
print("best_params: ", rand_cv.best_params_)
print("best_score_: ", rand_cv.best_score_)

C:\Users\vbak0\miniforge3\envs\fintech\lib\site-packages\xgboost\core.py:158: UserWarning: [16:27:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\vbak0\miniforge3\envs\fintech\lib\site-packages\xgboost\core.py:158: UserWarning: [16:27:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\vbak0\miniforge3\envs\fintech\lib\site-packages\xgboost\core.py:158: UserWarning: [16:27:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\vbak0\miniforge3

best_params:  {'scale_pos_weight': 3, 'n_estimators': 1000, 'max_depth': 1, 'learning_rate': 0.5, 'booster': 'gbtree'}
best_score_:  0.9191755229522637


In [157]:
from sklearn.metrics import roc_auc_score

In [159]:
xgb2 = XGBClassifier(n_jobs=-1, random_state=10, scale_pos_weight=3, n_estimators=1000,
                    max_depth=1, learning_rate=0.5, booster='gbtree')

xgb2.fit(X_train, y_train)
pred = xgb2.predict(X_test)
print(classification_report(y_test, pred))
print(roc_auc_score(y_test, pred))

              precision    recall  f1-score   support

       False       0.94      0.82      0.87     11706
        True       0.61      0.85      0.71      3990

    accuracy                           0.82     15696
   macro avg       0.78      0.83      0.79     15696
weighted avg       0.86      0.82      0.83     15696

0.8331782171985577


# 데이터 분석 과정 재정리
* 1) 데이터 선택 및 로딩
* 2) 결측값, 이상값 탐지
* 3) EDA( 탐색적 데이터 분석 )
* 4) 변수 데이터 타입, 서열변수, 카테고리 변수 처리
* 5) 변수 선택, 파생변수 생성
* 6) 홀드아웃(훈련데이터, 테스트 데이터로 분리)
* 7) 분석에 맞는 알고리즘 불러와서 모델 생성
* 8) 분석 목적에 맞는 지표를 이용해서 모델 성능 검증
* 9) 하이퍼파라미터 튜닝(클래스 불균형 해소 - 모델에서 소수 클래스 가중치 조절, 데이터 증폭/축소)
* 10) 최종 모델 선택